In [1]:
import sys
import pandas as pd
import numpy as np
import torch
from torch import Tensor

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-11 22:42:32,430 log.py               31   [INFO] [__init__] skip wandb init 


In [15]:
import pickle
# prep_df = pd.read_csv("../data/preprocessed/train_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_True_False_False_False.csv")
# with open("../data/preprocessed/train_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_True_False_False_False.pkl", "rb") as fin:
#     prep_df = pickle.load(fin)
prep_df = dr.load_preprocessed_df(
          dataset_name="train",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV6",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

prep_df

2021-11-11 22:55:02,254 data_repository.py   253  [INFO] [load_preprocessed_df] loading data/preprocessed/train_BaselineKernelPreprocessorV6_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-11 22:55:05,631 data_repository.py   260  [INFO] [load_preprocessed_df] done. 


,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,3,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,5
14184,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,5,351,352,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5
14185,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,5,101,102,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5
14186,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,20

In [4]:
prep_df["answer_text_count"] = prep_df.apply(lambda row: row["context"].count(row["answer_text"]), axis=1)

In [5]:
prep_df.drop_duplicates("id")["answer_text_count"].value_counts().sort_index()

1      652
2      174
3       65
4       52
5       29
6       16
7       14
8       14
9        6
10      14
11       5
12       5
13       3
14       3
15       7
16       2
17       2
18       9
19       1
20       2
22       2
23       7
24       2
26       2
27       1
29       1
30       2
31       1
33       1
38       1
40       1
41       1
43       2
44       1
46       2
53       1
60       1
62       1
85       3
86       1
96       1
102      1
110      2
155      1
Name: answer_text_count, dtype: int64

In [6]:
prep_df.drop_duplicates("id").query("answer_text_count == 155")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,answer_text_count
10139,69bce5adf,इन्द्र (या इंद्र) हिन्दू धर्म में सभी देवताओं ...,हिंदू धर्म के अनुसार देवताओं का राजा कौन है?,इन्द्र,0,hindi,"[0, 101195, 15858, 287, 15149, 175478, 11846, ...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,6,155


In [7]:
best_checkpoints = dr.best_checkpoint_filepaths("e042")
checkpoint = dr.load_checkpoint_from_filepath(filepath_from_root=best_checkpoints[0], load_from_gcs=True, rm_local_after_load=True) 

2021-11-11 22:46:00,322 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e042/best_checkpoint/0_3_1.1404_0.6902.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e042/best_checkpoint/0_3_1.1404_0.6902.pkl 
2021-11-11 22:46:54,563 repository.py        176  [INFO] [__download_from_gcs] download done. 


In [37]:
ids = checkpoint.val_pospro_ids
answer_preds = checkpoint.val_pospro_answer_preds
answer_texts = checkpoint.val_pospro_answer_texts
jaccards = [jaccard(answer_pred, answer_text) for answer_pred, answer_text in zip(answer_preds, answer_texts)]

fold_df = pd.DataFrame()
fold_df["id"] = ids
fold_df["answer_pred"] = answer_preds
fold_df["answer_text"] = answer_texts
fold_df["jaccard"] = jaccards

fold_df = fold_df.merge(prep_df[["id", "language"]].drop_duplicates(), on="id", how="left")
fold_df

,id,answer_pred,answer_text,jaccard,language
0,01a91f15a,"1,250",सभी पक्षियो के 12.6%,0.000000,hindi
1,01ae1bdd3,वैज्ञानिक,भारतीय वैज्ञानिक,0.500000,hindi
2,02c482c70,अरस्तू,अरस्तू,1.000000,hindi
3,036947361,"டிசம்பர் 25, 1642","டிசம்பர் 25, 1642",1.000000,tamil
4,047e3b82d,\n1988,19 मार्च 1882,0.000000,hindi
5,063be4c5a,मैथिलीशरण गुप्त,मैथिलीशरण गुप्त,1.000000,hindi
6,09ce55b23,ஆனி ஹதாவேயை,18,0.000000,tamil
7,0e0587b16,विंडहॉक,विंडहॉक,1.000000,hindi
8,0e4f1c353,ஆறு,ஆறு,1.000000,tamil
9,10e0404a2,मोहम्मद बिन सलमान अल सऊद,मोहम्मद बिन सलमान अल सऊद,1.000000,hindi


In [38]:
fold_df = fold_df.merge(prep_df.drop_duplicates("id")[["id", "answer_text_count", "question", "context"]], on="id", how="left")

In [27]:
fold_df.sort_values("answer_text_count", ascending=False)

,id,answer_pred,answer_text,jaccard,language,answer_text_count,question
210,f18b5f1c5,கொலம்பசு,கொலம்பசு,1.000000,tamil,53,அமெரிக்காவைக் கண்டுபிடித்தவர் யார்?
56,55323822a,வெள்ளி,வெள்ளி,1.000000,tamil,46,வெப்பமான கிரகம் எது?
195,e51cd0595,कन्नड़,कन्नड़,1.000000,hindi,43,मैसूर शहर की आधिकारिक भाषा क्या है?
123,95f1f21cb,दो,दो,1.000000,hindi,38,मंगल ग्रह में कितने चंद्रमा हैं?
108,8203b360a,सेंट्रल थाई,थाई,0.500000,hindi,30,थाईलैंड की आधिकारिक भाषा क्या है?
46,41660850a,தாலாட்டு,தாலாட்டு,1.000000,tamil,27,தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...
184,d825b56bb,लक्ष्मी,लक्ष्मी,1.000000,hindi,26,"हिन्दू धर्म में धन, सम्पदा, शान्ति और समृद्धि ..."
92,7531d7e07,बैडमिंटन,बैडमिंटन,1.000000,hindi,24,"साइना नेहवाल, किस खेल से सम्बंधित खिलाड़ी है?"
79,6d57418b8,"ஆகஸ்ட் 14, 1945",1945,0.333333,tamil,23,இரண்டாம் உலகப் போர் எப்போது முடிந்தது?
146,af2573e66,भारत,भारत,1.000000,hindi,22,किस देश ने शतरंज के गेम का अविष्कार किया था?


In [34]:
print(fold_df.query("id == '80452bc4d'").question.iloc[0])

உலகின் மிகப்பெரிய மனிதனால் உருவாக்கப்பட்ட கால்வாய் எது?


In [39]:
print(fold_df.query("id == '80452bc4d'").context.iloc[0])

கால்வாய் எனப்படுவது நீர்ப்பாசனத்துக்காக கால்வாய்கள், கப்பல் போக்குவரத்துக்கான கால்வாய்கள் என இருவகைப்படும். போக்குவரத்துக்கான கால்வாய்கள் செயற்கையாக அமைக்கப்படும் நீரிணைகள் ஆகும். இரு கடற்பகுதிகள், ஆற்றுப் பகுதிகள், அல்லது ஏரிப் பகுதிகளை இணைப்பதாக வெட்டப்படும் இவை கடற்போக்குவரத்துத் தூரத்தைக் குறைப்பதை நோக்கமாகக் கொண்டு வெட்டப்படுபவை ஆகும்.
சுயஸ் கால்வாய், பனாமாக் கால்வாய் போன்றன கப்பற் போக்குவரத்துக்காக வெட்டப்பட்ட குறிப்பிடத்தக்க கால்வாய்களாகும்.
வரலாறு கால்நடை விலங்கால் செலுத்தப்படும் வண்டி போக்குவரத்து திறன் ஒரு குறிப்பிட்ட வரையறைக்குள் தான் இருந்தது. கோவேறு கழுதை பூட்டப்பட்ட வண்டியில் அதிகபட்சம் 8 டன் [1] 250 pounds (113kg) என்ற அளவில் உள்ள எடைகொண்ட சரக்குப் பொருட்களை சில நாட்களுக்கோ அல்லது வாரங்களுக்கோ சுமக்க முடியும்.[1] இது மிகச் சிறிய இடைவெளிகளுக்கும், காலத்திற்கும் பொருத்தமானது தவிர, வண்டிகள் செல்ல சாலைகள் தேவை. இதற்கு பதிலாக பழங்காலத்தில் எளிமையான, மலிவான போக்குவரத்திற்கு கால்வாய் பொருத்தமானதாக இருந்திருக்கிறது. பண்டைய வரலாறு அறியப்பட்ட பழமையான கால்வாய்களில் முதன்மையானது நீர

In [35]:
print(fold_df.query("id == 'e0090c270'").question.iloc[0])

உலகின் மிகப்பெரிய கால்வாய் எது?


In [40]:
print(fold_df.query("id == 'e0090c270'").context.iloc[0])

கால்வாய் எனப்படுவது நீர்ப்பாசனத்துக்காக கால்வாய்கள், கப்பல் போக்குவரத்துக்கான கால்வாய்கள் என இருவகைப்படும். போக்குவரத்துக்கான கால்வாய்கள் செயற்கையாக அமைக்கப்படும் நீரிணைகள் ஆகும். இரு கடற்பகுதிகள், ஆற்றுப் பகுதிகள், அல்லது ஏரிப் பகுதிகளை இணைப்பதாக வெட்டப்படும் இவை கடற்போக்குவரத்துத் தூரத்தைக் குறைப்பதை நோக்கமாகக் கொண்டு வெட்டப்படுபவை ஆகும்.
சுயஸ் கால்வாய், பனாமாக் கால்வாய் போன்றன கப்பற் போக்குவரத்துக்காக வெட்டப்பட்ட குறிப்பிடத்தக்க கால்வாய்களாகும்.
வரலாறு கால்நடை விலங்கால் செலுத்தப்படும் வண்டி போக்குவரத்து திறன் ஒரு குறிப்பிட்ட வரையறைக்குள் தான் இருந்தது. கோவேறு கழுதை பூட்டப்பட்ட வண்டியில் அதிகபட்சம் 8 டன் [1] 250 pounds (113kg) என்ற அளவில் உள்ள எடைகொண்ட சரக்குப் பொருட்களை சில நாட்களுக்கோ அல்லது வாரங்களுக்கோ சுமக்க முடியும்.[1] இது மிகச் சிறிய இடைவெளிகளுக்கும், காலத்திற்கும் பொருத்தமானது தவிர, வண்டிகள் செல்ல சாலைகள் தேவை. இதற்கு பதிலாக பழங்காலத்தில் எளிமையான, மலிவான போக்குவரத்திற்கு கால்வாய் பொருத்தமானதாக இருந்திருக்கிறது. பண்டைய வரலாறு அறியப்பட்ட பழமையான கால்வாய்களில் முதன்மையானது நீர

In [32]:
for i in [1, 2, 3, 4, 5, 10, 15, 20, 25, 30]:
    print(i, fold_df.query(f"answer_text_count > {i:.3f}").jaccard.mean(), fold_df.query(f"answer_text_count <= {i:.3f}").jaccard.mean())

1 0.716140350877193 0.6710255456349208
2 0.6551515151515152 0.7017337490551776
3 0.6782608695652174 0.6933593399695095
4 0.6533333333333332 0.6971166835528538
5 0.6408602150537634 0.6982184193121693
10 0.8074074074074074 0.6799574138598529
15 0.8022222222222223 0.6821695665445665
20 0.8833333333333334 0.6811796706162904
25 0.9285714285714286 0.6825213109935333
30 1.0 0.6845872291077771


## 同じ context と同じ question と同じ answer_text を見てみる

In [8]:
prep_df.drop_duplicates("id").context.value_counts().head(20).values

array([6, 6, 5, 5, 5, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [58]:
prep_df.drop_duplicates("id").answer_text.value_counts().head(20)

दो                          6
तीन                         6
1945                        5
29                          4
महाराष्ट्र                  4
ஆப்பிரிக்கா                 4
24                          3
होली                        3
இஸ்லாமாபாத்                 3
1000                        3
1949                        3
१४,९६,००,००० किलोमीटर       3
नई दिल्ली                   3
7 अप्रैल 1948               3
இந்தியா                     3
18                          3
चार्ल्स बैबेज               3
லண்டன்                      2
1920                        2
1917ஆம் ஆண்டு நவம்பர் 19    2
Name: answer_text, dtype: int64

In [63]:
prep_df.drop_duplicates("id").query("answer_text == 'दो'")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,answer_text_count
7885,f87b3f859,शतरंज (चैस) दो खिलाड़ियों के बीच खेला जाने वाल...,शतरंज गेम में कितने खिलाडी होते है?,दो,12,hindi,"[0, 8933, 21100, 36860, 156180, 421, 180513, 1...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,4,23,23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4,24
7916,3e2e9a901,"मानव शरीर रचना विज्ञान में, आंत (या अंतड़ी) आह...",मानव के शरीर में सम्पूर्ण कितने आंत होती है?,दो,142,hindi,"[0, 37535, 287, 31991, 421, 99476, 180513, 222...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,7,54,54,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,7,5
7994,2c789f525,चुंबकीय क्षेत्र विद्युत धाराओं और चुंबकीय सामग...,एक चुंबक के कितने पक्ष होते हैं?,दो,106,hindi,"[0, 967, 36913, 43753, 1223, 287, 180513, 2070...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,46,46,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5,7
7996,d2aed13da,किसी राज्य द्वारा व्यक्तियों या विविध संस्था स...,टैक्स कितने प्रकार के होते है?,दो,278,hindi,"[0, 186213, 180513, 19440, 287, 8358, 460, 32,...","[None, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,4,87,87,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4,2
11163,f3e4ced27,भगवान राम चन्द्र जी का जीवनकाल एवं पराक्रम महर...,भगवान राम को कुल कितने पुत्र थे?,दो,243,hindi,"[0, 59351, 14131, 629, 40696, 180513, 98456, 1...","[None, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,79,79,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5,12
12810,95f1f21cb,मंगल सौरमंडल में सूर्य से चौथा ग्रह है। पृथ्वी...,मंगल ग्रह में कितने चंद्रमा हैं?,दो,145,hindi,"[0, 164922, 6, 49034, 421, 180513, 86024, 411,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,57,57,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5,38


In [55]:
prep_df.drop_duplicates("id").context.value_counts().head(20)

யானை பாலூட்டி வகையைச் சேர்ந்த ஒரு தாவர உண்ணி (இலையுண்ணி) விலங்காகும். இது நிலத்தில் வாழும் விலங்குகள் யாவற்றினும் மிகப் பெரியதாகும். மிக நீண்ட நாட்கள் வாழக்கூடியதும் ஆகும் (~ 70 ஆண்டுகள்). மனிதர்கள் தவிர்ந்த மற்றைய விலங்குகளில் இதுவே அதிக நாட்கள் வாழும் தரைவாழ் விலங்கு ஆகும். யானைகள் மிகவும் வலிமையானவை. வேட்டை விலங்குகளும் காட்டுயிர்களின் உணவுப் படிநிலையில் உயர்நிலையிலுள்ளவைகளுமான சிங்கம் (அரிமா), புலி முதலியனவும் நெருங்க முடியாத வலிமை கொண்டவை. சிங்கங்கள் ஒன்றாகச் சேர்ந்து, தனியாக வர நேரிடும் களைத்த அல்லது இளைத்த யானையைக் கொல்லும். ஆனால், இவ்வகை நிகழ்வுகள் மிகமிகக் குறைவே. யானைகள் குடும்பமாக வாழும். மேலும், இவை அதிக ஞாபக சக்தி கொண்டவை.\nயானைகளில் மூன்று சிற்றினங்கள் இன்று உலகில் எஞ்சியுள்ளன. அவை, ஆப்பிரிக்கப் புதர்வெளி யானைகள், ஆப்பிரிக்கக் காட்டு யானைகள், ஆசிய யானைகள் ஆகும். இவைகளுக்கிடையே சிறப்பான வேறுபாடுகள் உள்ளன. பொதுவாக, எல்லா யானைகளும் ஏறத்தாழ 70 ஆண்டுகள்வரை உயிர்வாழ்கின்றன.\nஆண் யானைக்குக் களிறு என்று பெயர். பெண் யானைக்குப் பிடி என்று பெயர். யானையின் குட்டியைக் கன்று என்றோ, குட

In [9]:
prep_df.drop_duplicates("id").question.value_counts().head(20)

இங்கிலாந்து நாட்டின் தலைநகரம் எது?                          2
மலேசியா நாட்டின் பரப்பளவு என்ன?                             2
நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?                        2
பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?                         2
சென்னை நகரம் எப்போது நிறுவப்பட்டது?                         2
ஆஸ்திரேலியாவில் எத்தனை மாநிலங்கள் உள்ளன?                    2
ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?                       2
உலகின் இரண்டாவது பெரிய கண்டம் எது?                          2
இந்திராகாந்தி எப்போது பிறந்தார்?                            2
பொருளாதாரத்தின் தந்தை என்று கருதப்படுபவர் யார்?             2
केनरा बैंक के संस्थापक कौन हैं?                             1
विनायक सावरकर का जन्म कब हुआ था?                            1
फिर भी दिल है हिंदुस्तानी फिल्म के निर्देशक कौन थे?         1
अमरीकी सेना द्वारा आतंकी ओसामा बिन लादेन को कब मारा गया?    1
மனித உடலில் எத்தனை எலும்புகள் உள்ளன?                        1
चेचक का वैज्ञानिक नाम क्या है?                              1
निकोला ट

In [62]:
prep_df.drop_duplicates("id").query("question == 'இங்கிலாந்து நாட்டின் தலைநகரம் எது?'")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,answer_text_count
4275,bd50e3092,இங்கிலாந்து ஐக்கிய இராச்சியத்திலுள்ள நான்கு நா...,இங்கிலாந்து நாட்டின் தலைநகரம் எது?,லண்டன்,542,tamil,"[0, 242389, 3686, 216652, 30346, 17056, 56120,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,2,161,163,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2,14
4787,2d4b48611,இங்கிலாந்து ஐக்கிய இராச்சியத்திலுள்ள நான்கு நா...,இங்கிலாந்து நாட்டின் தலைநகரம் எது?,லண்டன்,542,tamil,"[0, 242389, 3686, 216652, 30346, 17056, 56120,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,2,160,162,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2,14


In [43]:
prep_df.drop_duplicates("id").query("question == 'மலேசியா நாட்டின் பரப்பளவு என்ன?'")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,answer_text_count
4709,ae3fbd19f,"மலேசியா (Malaysia), தென்கிழக்காசியாவில் உள்ள ஒ...",மலேசியா நாட்டின் பரப்பளவு என்ன?,"329,847",1067,tamil,"[0, 9696, 44628, 8040, 6490, 216652, 58523, 71...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,2,355,357,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2,1
4826,34ed796e6,"மலேசியா (Malaysia), தென்கிழக்காசியாவில் உள்ள ஒ...",மலேசியா நாட்டின் பரப்பளவு என்ன?,"329,847 சதுர கிலோமீட்டர்கள்",1067,tamil,"[0, 9696, 44628, 8040, 6490, 216652, 58523, 71...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,2,355,364,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2,1


In [45]:
prep_df.drop_duplicates("id").query("question == 'நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?'")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,answer_text_count
2716,bc05d5d9d,நெல் அல்லது அரிசி (ஒலிப்பு) (rice) என்பது புல்...,நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?,Oryza sativa,528,tamil,"[0, 60070, 3769, 19376, 39290, 21651, 161902, ...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,1,165,169,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1,3
3772,eaa4b6fec,நெல் அல்லது அரிசி (ஒலிப்பு) (rice) என்பது புல்...,நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?,Oryza sativa,528,tamil,"[0, 60070, 3769, 19376, 39290, 21651, 161902, ...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,1,165,169,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1,3


In [48]:
prep_df.drop_duplicates("id").query("question == 'பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?'")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,answer_text_count
4140,6727e0764,"பாக்கித்தான் (Pakistan, பாகிஸ்தான், (pækɨstæn ...",பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?,இஸ்லாமாபாத்,283,tamil,"[0, 10897, 17708, 4575, 9245, 216652, 30346, 1...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,1,107,110,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1,2
5081,e6ec0096f,"பாக்கித்தான் (Pakistan, பாகிஸ்தான், ( அல்லது p...",பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?,இஸ்லாமாபாத்,232,tamil,"[0, 10897, 17708, 4575, 9245, 216652, 30346, 1...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,1,90,93,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1,2


In [53]:
print(prep_df.drop_duplicates("id").query("question == 'பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?'").iloc[0]["context"])

பாக்கித்தான் (Pakistan, பாகிஸ்தான், (pækɨstæn அல்லது pɑːkiˈstɑːn; Urdu: پاکستان‎), அதிகாரபூர்வமாக பாக்கிஸ்தான் இசுலாமியக் குடியரசு (Urdu: اسلامی جمہوریۂ پاکستان‎), ஆசிய கண்டத்தில் உள்ள ஒரு நாடாகும். மக்கள் தொகை அடர்த்தி மிகுந்த நாடுகளில் பாக்கிஸ்தானும் ஒன்று. பாக்கிஸ்தானின் தலைநகர் இஸ்லாமாபாத். கராச்சி முக்கிய துறைமுகமும் தொழில் நகரமும் ஆகும். இந்திய எல்லையின் அருகில் உள்ள லாகூர் மற்றொரு முக்கிய நகரம்.
180 மில்லியன் மக்கள்தொகை கொண்ட பாக்கிஸ்தான் ஆறாவது மிகுந்த மக்கள்தொகையுடைய நாடாகும். 796,095கிமீ2 (307,374 ச மை) பரப்பளவுள்ள இந்த நாடு இதனடிப்படையில்  36வது பெரிய நாடாக விளங்குகின்றது. தெற்கில் அரபிக்கடல் மற்றும் ஓமன் குடாவில் 1,046-kilometre (650mi) தொலைவுள்ள கடற்கரையைக் கொண்டுள்ளது; கிழக்கில் இந்தியாவும் மேற்கில் ஆப்கானிஸ்தானும் தென்மேற்கில் ஈரானும் வடகிழக்குக் கோடியில் சீன மக்கள் குடியரசும் எல்லைகளாக அமைந்துள்ளன. வடக்கில் ஆப்கானிஸ்தானின் குறுகிய வாகான்  இடைப்பகுதி</i>யால்  தஜிகிஸ்தானிலிருந்து பிரிக்கப்பட்டுள்ளது. தவிரவும் தனது கடல் எல்லையை ஓமனுடன் பகிர்ந்து கொண்டுள்ளது.
தற்போது பாக்கி

In [54]:
print(prep_df.drop_duplicates("id").query("question == 'பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?'").iloc[1]["context"])

பாக்கித்தான் (Pakistan, பாகிஸ்தான், ( அல்லது pɑːkiˈstɑːn; ), அதிகாரபூர்வமாக பாக்கிஸ்தான் இசுலாமியக் குடியரசு (), ஆசிய கண்டத்தில் உள்ள ஒரு நாடாகும். மக்கள் தொகை அடர்த்தி மிகுந்த நாடுகளில் பாக்கிஸ்தானும் ஒன்று. பாக்கிஸ்தானின் தலைநகர் இஸ்லாமாபாத். கராச்சி முக்கிய துறைமுகமும் தொழில் நகரமும் ஆகும். இந்திய எல்லையின் அருகில் உள்ள லாகூர் மற்றொரு முக்கிய நகரம்.
180 மில்லியன் மக்கள்தொகை கொண்ட பாக்கிஸ்தான் ஆறாவது மிகுந்த மக்கள்தொகையுடைய நாடாகும். 796,095கிமீ2 () பரப்பளவுள்ள இந்த நாடு இதனடிப்படையில்  36வது பெரிய நாடாக விளங்குகின்றது. தெற்கில் அரபிக்கடல் மற்றும் ஓமன் குடாவில்  தொலைவுள்ள கடற்கரையைக் கொண்டுள்ளது; கிழக்கில் இந்தியாவும் மேற்கில் ஆப்கானிஸ்தானும் தென்மேற்கில் ஈரானும் வடகிழக்குக் கோடியில் சீன மக்கள் குடியரசும் எல்லைகளாக அமைந்துள்ளன. வடக்கில் ஆப்கானிஸ்தானின் குறுகிய வாகான்  இடைப்பகுதி</i>யால்  தஜிகிஸ்தானிலிருந்து பிரிக்கப்பட்டுள்ளது. தவிரவும் தனது கடல் எல்லையை ஓமனுடன் பகிர்ந்து கொண்டுள்ளது.
தற்போது பாக்கிஸ்தானாக அறியப்படும் பகுதியில் பல தொன்மையான நாகரிகங்கள் தழைத்துள்ளன.  புதிய கற்காலத்தின

In [51]:
prep_df.query("id == '6727e0764'").context.iloc[0].find("இஸ்லாமாபாத்")

283

In [52]:
prep_df.query("id == 'e6ec0096f'").context.iloc[0].find("இஸ்லாமாபாத்")

232

#### 集計してみる

In [16]:
prep_df = prep_df.merge(prep_df.drop_duplicates("id").context.value_counts().rename_axis("context").rename("context_count"), on="context", how="left")
prep_df = prep_df.merge(prep_df.drop_duplicates("id").question.value_counts().rename_axis("question").rename("question_count"), on="question", how="left")
prep_df = prep_df.merge(prep_df.drop_duplicates("id").answer_text.value_counts().rename_axis("answer_text").rename("answer_text_count"), on="answer_text", how="left")
prep_df

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,context_count,question_count,answer_text_count
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,3,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3,2,1,2
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,2,1,2
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,2,1,2
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,2,1,2
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,5,1,1,1
14184,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,5,351,352,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5,1,1,1
14185,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,5,101,102,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5,1,1,1
14186,

In [18]:
prep_df.query('context_count == 6').drop_duplicates("id").answer_text

620                           பிடி
838                    70 ஆண்டுகள்
972                             70
1126                             3
2194                   22 மாதங்கள்
2418           மூன்று சிற்றினங்கள்
6358    चार्ल्स लुई अल्फोंस लैवेरन
6642                         कुनैन
6788    चार्ल्स लुई अल्फोंस लैवेरन
6991                    एनोफ़िलेज़
7389                    प्रोटोज़ोआ
7640             लाल रक्त कोशिकाओं
Name: answer_text, dtype: object

In [20]:
for q in prep_df.query('context_count == 6').drop_duplicates("id").question:
    print(q)

பெண் யானை தமிழ் மொழியில் எவ்வாறு அழைக்கப்படுகிறது?
யானையின் சராசரி ஆயுட்காலம் எவ்வளவு?
ஆப்பிரிக்க யானைகளின் சராசரி ஆயுட்காலம் என்ன?
ஆண் யானை சராசரி எத்தனை மீட்டர் உயரம் இருக்கும்
யானையின் கர்ப்பகாலம் எத்தனை நாட்கள்?
எத்தனை யானை இனங்கள் உள்ளன?
किसने अल्जीरिया में काम करते हुए पहली बार लाल रक्त कोशिका के अन्दर परजीवी को देखा था?
मलेरिया संक्रमण का इलाज किस दवा से किया जाता?
दुनिया में सबसे पहले मलेरिया के बारे में किसने खोजा था?
मलेरिया किस प्रकार के मच्छर से फैलता है?
मलेरिया किसके द्वारा फैलता है?
मलेरिया सबसे पहले किस मानव अंग पर असर करता है?


In [22]:
prep_df.drop_duplicates("id").question_count.value_counts()

1    1094
2      20
Name: question_count, dtype: int64

In [24]:
prep_df.query('question_count == 2').drop_duplicates("id").sort_values("context").answer_text

2371                               ஆறு
884                                ஆறு
4787                            லண்டன்
4275                            லண்டன்
3465          1917ஆம் ஆண்டு நவம்பர் 19
4013          1917ஆம் ஆண்டு நவம்பர் 19
3404                       ஆப்பிரிக்கா
5198                       ஆப்பிரிக்கா
5011     1639ஆம் ஆண்டு ஆகஸ்ட் மாதம் 22
3347     1639ஆம் ஆண்டு ஆகஸ்ட் மாதம் 22
2716                      Oryza sativa
3772                      Oryza sativa
5081                       இஸ்லாமாபாத்
4140                       இஸ்லாமாபாத்
4849                      ஆடம் இசுமித்
4875                       ஆடம் சிமித்
4826       329,847 சதுர கிலோமீட்டர்கள்
4709                           329,847
3193    மேரிகியூரி மற்றும் பியரிகியூரி
3215                              1898
Name: answer_text, dtype: object

In [28]:
20/1114

0.017953321364452424

In [25]:
prep_df.loc[3193].question

'ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?'

In [26]:
prep_df.loc[3215].question

'ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?'

In [33]:
prep_df.query('context_count == 5').sort_values(["context", "question"]).drop_duplicates("id").answer_text

7261                       98%
6395     140 लाख वर्ग किलोमीटर
7800          14,000,000 किमी2
7499    जॉन जॉर्ज बार्थोलोम्यू
7671            15 फ़रवरी 2010
7961           वाट्सन और क्रिक
6929                रॉबर्ट हूक
8150            कोशिका विज्ञान
8203    एंटोनी वॉन ल्यूवेन्हॉक
7455                रॉबर्ट हूक
7924                       लाल
7097                   लाल रंग
6348            करीब पाँच लिटर
7483                       लाल
8049                 पाँच लिटर
Name: answer_text, dtype: object

In [34]:
for q in prep_df.query('context_count == 5').sort_values(["context", "question"]).drop_duplicates("id").question:
    print(q)

अंटार्कटिका का कितना भाग औसतन मोटी बर्फ से आच्छादित है?
अंटार्कटिका का क्षेत्र क्या है?
अंटार्कटिका का क्षेत्रफल कितना है?
अंटार्कटिका नाम का पहले पहल औपचारिक प्रयोग किसने किया था?
क्रायो जैव विज्ञान प्रयोगशाला का कब उद्घाटन किया गया था?
किसने डीएनए के डबल-हेलिक्स संरचना की पहली बार घोषणा की?
कोशिका की खोज किसने की?
कोशिकाओं का विधिवत अध्ययन क्या कहलाता है?
जीवित कोशिका को सर्वप्रथम किसने देखा था?
१८३१ में किसने कोशिका में केंद्रक एवं केंद्रिका का पता लगाया?
मानव के खून का रंग क्या होता है?
मानव के लहू का रंग कौनसा होता है?
मानव के शरीर में कितने प्रतिशत लहू है?
मानव के शरीर में लहू का रंग क्या है?
मानव शरीर में कितना खून होता है?


In [35]:
import Levenshtein

In [38]:
Levenshtein.distance("मानव के खून का रंग क्या होता है?", "मानव के लहू का रंग कौनसा होता है?")

6

In [58]:
from tqdm.auto import tqdm

distances = []
for i, row in tqdm(prep_df.drop_duplicates("id").iterrows(), total=prep_df.id.nunique()):
    for j, row2 in prep_df.drop_duplicates("id").iterrows():
        if j <= i:
            continue
        distances.append(
            {
                "id1": row["id"],
                "id2": row2["id"],
                "question1": row["question"],
                "question2": row2["question"],
                "answer_text1": row["answer_text"],
                "answer_text2": row2["answer_text"],
                # "leven_context_distance": Levenshtein.distance(row["context"], row2["context"]),
                "leven_question_distance": Levenshtein.distance(row["question"], row2["question"]),
                "leven_answer_text_distance": Levenshtein.distance(row["answer_text"], row2["answer_text"]),
                # "jaccard_context_distance": jaccard(row["context"], row2["context"]),
                "jaccard_question_distance": jaccard(row["question"], row2["question"]),
                "jaccard_answer_text_distance": jaccard(row["answer_text"], row2["answer_text"]),
            }
        )

  0%|          | 0/1114 [00:00<?, ?it/s]

In [60]:
distance_df = pd.DataFrame(distances)# .sort_values("distance").reset_index(drop=True)
distance_df

,id1,id2,question1,question2,answer_text1,answer_text2,leven_question_distance,leven_answer_text_distance,jaccard_question_distance,jaccard_answer_text_distance
0,903deec17,d9841668c,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,காளிதாசன் எங்கு பிறந்தார்?,206,காசுமீரில்,26,10,0.000000,0.0
1,903deec17,29d154b56,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,பென்சிலின் கண்டுபிடித்தவர் யார்?,206,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,27,26,0.000000,0.0
2,903deec17,41660850a,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,206,தாலாட்டு,57,8,0.000000,0.0
3,903deec17,b29c82c22,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,பூமியின் அருகில் உள்ள விண்மீன் எது?,206,சூரியனும்,28,9,0.000000,0.0
4,903deec17,d419db018,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?,206,IBM,37,3,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
619936,31179f1bb,7f997884d,नित्यशास्त्र किसने लिखा है?,मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,भरत मुनि,"२७ अक्तूबर, १६०५",37,14,0.000000,0.0
619937,31179f1bb,ee569fe45,नित्यशास्त्र किसने लिखा है?,लीजा रे की पहली फिल्म का नाम क्या था?,भरत मुनि,नेताजी,29,7,0.000000,0.0
619938,0d35dc007,7f997884d,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,17,"२७ अक्तूबर, १६०५",38,16,0.058824,0.0
619939,0d35dc007,ee569fe45,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,लीजा रे की पहली फिल्म का नाम क्या था?,17,नेताजी,35,6,0.062500,0.0


In [61]:
distance_df.sort_values("leven_question_distance").head(100)

,id1,id2,question1,question2,answer_text1,answer_text2,leven_question_distance,leven_answer_text_distance,jaccard_question_distance,jaccard_answer_text_distance
73515,0e4f1c353,8d15b9e7a,ஆஸ்திரேலியாவில் எத்தனை மாநிலங்கள் உள்ளன?,ஆஸ்திரேலியாவில் எத்தனை மாநிலங்கள் உள்ளன?,ஆறு,ஆறு,0,0,1.000000,1.000000
252317,3d205a22a,701861590,இந்திராகாந்தி எப்போது பிறந்தார்?,இந்திராகாந்தி எப்போது பிறந்தார்?,1917ஆம் ஆண்டு நவம்பர் 19,1917ஆம் ஆண்டு நவம்பர் 19,0,0,1.000000,1.000000
285822,bd50e3092,2d4b48611,இங்கிலாந்து நாட்டின் தலைநகரம் எது?,இங்கிலாந்து நாட்டின் தலைநகரம் எது?,லண்டன்,லண்டன்,0,0,1.000000,1.000000
234939,bc9f0d533,5179ed725,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,மேரிகியூரி மற்றும் பியரிகியூரி,1898,0,30,1.000000,0.000000
282569,6727e0764,e6ec0096f,பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?,பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?,இஸ்லாமாபாத்,இஸ்லாமாபாத்,0,0,1.000000,1.000000
210039,bc05d5d9d,eaa4b6fec,நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?,நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?,Oryza sativa,Oryza sativa,0,0,1.000000,1.000000
246358,34846a420,b35c26634,சென்னை நகரம் எப்போது நிறுவப்பட்டது?,சென்னை நகரம் எப்போது நிறுவப்பட்டது?,1639ஆம் ஆண்டு ஆகஸ்ட் மாதம் 22,1639ஆம் ஆண்டு ஆகஸ்ட் மாதம் 22,0,0,1.000000,1.000000
248096,715de8bbe,6f8169ceb,உலகின் இரண்டாவது பெரிய கண்டம் எது?,உலகின் இரண்டாவது பெரிய கண்டம் எது?,ஆப்பிரிக்கா,ஆப்பிரிக்கா,0,0,1.000000,1.000000
309869,ae3fbd19f,34ed796e6,மலேசியா நாட்டின் பரப்பளவு என்ன?,மலேசியா நாட்டின் பரப்பளவு என்ன?,"329,847","329,847 சதுர கிலோமீட்டர்கள்",0,20,1.000000,0.333333
316132,67d7a612e,0745249f0,பொருளாதாரத்தின் தந்தை என்று கருதப்படுபவர் யார்?,பொருளாதாரத்தின் தந்தை என்று கருதப்படுபவர் யார்?,ஆடம் இசுமித்,ஆடம் சிமித்,0,2,1.000000,0.333333


In [63]:
distance_df.sort_values("jaccard_question_distance", ascending=False).head(100)

,id1,id2,question1,question2,answer_text1,answer_text2,leven_question_distance,leven_answer_text_distance,jaccard_question_distance,jaccard_answer_text_distance
285822,bd50e3092,2d4b48611,இங்கிலாந்து நாட்டின் தலைநகரம் எது?,இங்கிலாந்து நாட்டின் தலைநகரம் எது?,லண்டன்,லண்டன்,0,0,1.000000,1.000000
210039,bc05d5d9d,eaa4b6fec,நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?,நெல் தாவரத்தின் அறிவியல் பெயர் என்ன?,Oryza sativa,Oryza sativa,0,0,1.000000,1.000000
309869,ae3fbd19f,34ed796e6,மலேசியா நாட்டின் பரப்பளவு என்ன?,மலேசியா நாட்டின் பரப்பளவு என்ன?,"329,847","329,847 சதுர கிலோமீட்டர்கள்",0,20,1.000000,0.333333
246358,34846a420,b35c26634,சென்னை நகரம் எப்போது நிறுவப்பட்டது?,சென்னை நகரம் எப்போது நிறுவப்பட்டது?,1639ஆம் ஆண்டு ஆகஸ்ட் மாதம் 22,1639ஆம் ஆண்டு ஆகஸ்ட் மாதம் 22,0,0,1.000000,1.000000
316132,67d7a612e,0745249f0,பொருளாதாரத்தின் தந்தை என்று கருதப்படுபவர் யார்?,பொருளாதாரத்தின் தந்தை என்று கருதப்படுபவர் யார்?,ஆடம் இசுமித்,ஆடம் சிமித்,0,2,1.000000,0.333333
252317,3d205a22a,701861590,இந்திராகாந்தி எப்போது பிறந்தார்?,இந்திராகாந்தி எப்போது பிறந்தார்?,1917ஆம் ஆண்டு நவம்பர் 19,1917ஆம் ஆண்டு நவம்பர் 19,0,0,1.000000,1.000000
73515,0e4f1c353,8d15b9e7a,ஆஸ்திரேலியாவில் எத்தனை மாநிலங்கள் உள்ளன?,ஆஸ்திரேலியாவில் எத்தனை மாநிலங்கள் உள்ளன?,ஆறு,ஆறு,0,0,1.000000,1.000000
282569,6727e0764,e6ec0096f,பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?,பாக்கிஸ்தான் நாட்டின் தலைநகரம் எது?,இஸ்லாமாபாத்,இஸ்லாமாபாத்,0,0,1.000000,1.000000
248096,715de8bbe,6f8169ceb,உலகின் இரண்டாவது பெரிய கண்டம் எது?,உலகின் இரண்டாவது பெரிய கண்டம் எது?,ஆப்பிரிக்கா,ஆப்பிரிக்கா,0,0,1.000000,1.000000
234939,bc9f0d533,5179ed725,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,மேரிகியூரி மற்றும் பியரிகியூரி,1898,0,30,1.000000,0.000000
